In [41]:
import torch
from sklearn.metrics import DistanceMetric
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from scipy.spatial import distance

import sys
sys.path.append('/workspaces/dbm25/task_1_2')
from tabulate import tabulate


from extract_features import extract_features



In [42]:
def top_k_distance_search(image_path, k, feature_model, measure):
    """
    Finds the k most similar images to an input image based on a feature model and distance metric.
    
    Parameters:
    ----------
    image_path : str
        Path to the input image.
    k : int
        Number of similar images to return.
    feature_model : str
        Feature extraction model identifier to use.
    measure : str
        Distance metric name (compatible with sklearn.metrics.DistanceMetric).
    
    Returns:
    -------
    list of dict
        List of k dictionaries containing similar image information:
        - "image_name": image name
        - "file_path": image path
        - "class": image class
        - "distance_score": distance score (lower = more similar)
    """
    
    # Load pre-trained ResNet50 model
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
    model.eval()
    
    # Extract features from input image
    image_query_surrogate = extract_features(image_path, model)

    # Load feature space from database
    feature_space = torch.load("/workspaces/dbm25/data/extracted_features.pt")
    
    # Get and reshape (linearize) query image features
    image_query_feature = image_query_surrogate[feature_model].reshape(1,-1) 
    
    # Initialize distance metric
    dist = DistanceMetric.get_metric(measure)
    results = []

    # Compare query image against each image in the database
    for image_surrogate in feature_space:
        image_surrogate_feature = image_surrogate[feature_model].reshape(1,-1)
        
        # Calculate distance between query image and current database image
        distance_score = dist.pairwise(image_query_feature, image_surrogate_feature)[0][0]
        image_name = image_surrogate["file_path"].split("/").pop()

        new_result = {
            "image_name": image_name,
            "file_path": image_surrogate["file_path"],
            "class": image_surrogate["class"],
            "distance_score": distance_score
        }
        
        # Maintain a list of only the k most similar images
        if len(results) < k:
            results.append(new_result)
        else:
            # Sort by distance (descending - largest distance first)
            results.sort(key=lambda element: element["distance_score"], reverse=True)
            
            # Replace least similar image if current one is more similar
            if results[0]["distance_score"] > new_result["distance_score"]:
                results.pop(0)
                results.append(new_result)

    return results

In [43]:
def top_k_cosine_similarity_search(image_path, k, feature_model):
   """
    Finds the k most similar images to an input image based on a feature model and the cosine similarity.
    
    Parameters:
    ----------
    image_path : str
        Path to the input image.
    k : int
        Number of similar images to return.
    feature_model : str
        Feature extraction model identifier to use.
    
    Returns:
    -------
    list of dict
        List of k dictionaries containing similar image information:
        - "image_name": image name
        - "file_path": image path
        - "class": image class
        - "similarity_score": similarity score (higher = more similar)
    """
   
   # Load pre-trained ResNet50 model
   model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
   model.eval()
    
   # Extract features from input image
   image_query_surrogate = extract_features(image_path, model)
   

   # Load feature space from database
   feature_space = torch.load("/workspaces/dbm25/data/extracted_features.pt")
    
   # Get and reshape (linearize) query image features
   image_query_feature = image_query_surrogate[feature_model].reshape(1,-1) 
   
   results = []

   # Compare query image against each image in the database
   for image_surrogate in feature_space:
        image_surrogate_feature = image_surrogate[feature_model].reshape(1,-1)
        
        # Calculate distance between query image and current database image
        similarity_score = cosine_similarity(image_query_feature, image_surrogate_feature)[0][0]
        image_name = image_surrogate["file_path"].split("/").pop()

        new_result = {
            "image_name": image_name,
            "file_path": image_surrogate["file_path"],
            "class": image_surrogate["class"],
            "similarity_score": similarity_score
        }
        
        # Maintain a list of only the k most similar images
        if len(results) < k:
            results.append(new_result)
        else:
            # Sort by distance (descending - smallest similarity first)
            results.sort(key=lambda element: element["similarity_score"])
            
            # Replace least similar image if current one is more similar
            if results[0]["similarity_score"] < new_result["similarity_score"]:
                results.pop(0)
                results.append(new_result)

   return results


   

In [44]:
results = top_k_cosine_similarity_search(image_path="/workspaces/dbm25/data/Part1/Part1/brain_glioma/brain_glioma_0051.jpg", 
                                k=5, 
                                feature_model="cm",
                              )
results.reverse()

print(tabulate(results, headers='keys', tablefmt='psql'))


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Processing /workspaces/dbm25/data/Part1/Part1/brain_glioma/brain_glioma_0051.jpg


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 3 while Y.shape[1] == 300

In [49]:
# Load pre-trained ResNet50 model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
image_path="/workspaces/dbm25/data/Part1/Part1/brain_menin/brain_menin_0001.jpg"   

# Extract features from input image
feature_space = torch.load("/workspaces/dbm25/data/extracted_features.pt")

print(feature_space[0]["cm"].shape)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


torch.Size([300])
